# Putting it All Together

This notebook is a case study in working with python and several modules. It's a real problem I had to solve with real data. There are *many* ways to attack a problem such as this; this is simply one way. The point is to illustrate how you can get existing modules to do the heavy-lifting for you and that visualization is a powerful diagnostic tool. Try not to get caught up in the details of the model; it's quite complex and the point is not to understand all the equations, but the *procedure* of exploring data and fitting it to a model (read the citation if you're really interested all the gory details).

This notebook requires the following modules:
* `numpy`: dealing with arrays of numbers and mathematics
* `scipy`: collection of scientific algorithms
* `matplotlib`: de-facto plotting module
* `pandas`: module for organizing arrays of number into tables
* `bokeh`: another module for plotting, with emphasis on interactive visualization

The problem I needed to solve: predict the background sky brightness caused by the moon at a given location in the sky on a given date. This is to help plan observations at the telescope. As with all problems of this type, we need to do several things:

* Download/import/munge training data
* Model the training data
* Extract model parameters
* Graph the result(s) to see how well we do, maybe modify the model
* Use final model and parameters to make future predictions

### 1) The Data

In this case, the data to model is roughly 10 years of photometry from the Carnegie Supernova Project (CSP). Each and every measurement of the flux from a standard star has an associated estimate of the sky background (which must be subtracted from the raw counts of the star). These data were taken over many different times of the month and a many different sky altitudes, so are ideal for this problem.

Let's start by getting the data. For convenience, this has been included in the `data` folder and so we can load it up immediately into a `pandas` dataframe.

In [ ]:
import pandas as pd
data = pd.read_csv('data/skyfit.dat')

We can take a quick look at what's in this `DataFrame` by printing out the first few rows.

In [ ]:
print(data[0:10])

The column `jd` is the [Julian Day](https://en.wikipedia.org/wiki/Julian_day), a common numerical representation of the date, `RA` and `Decl` are the sky coordiates of the field, and `magsky` is the sky brighness. Let's have a look at the distribution of sky brightnesses to make sure they "make sense". The units should be magnitudes per square-arc-second and be on order of 22 or so, but should be smaller for bright time (full moon). Since we're just doing a quick-look, we can use `pandas`' built-in histogram plotter.

In [ ]:
%matplotlib inline
data.hist('magsky', bins=50)

As you can see, there is peak near 22 mag/square-arc-sec, as expected, but a broader peak at brighter backgrounds. We expect this is due to moonlight. Something to think about:  why would this be bi-modal?

We expect that the fuller the moon, the brighter it will be and the closer the observation is to the moon on the sky, the higher the background. So whatever model we use is going to require knowledge of the moon's position and phase. There are mathematical formulae for calculating these, but we'll use the handy `astropy.coordinates` module to do all the work for us. First, let's compute the lunar phase for each date in our table. To do this, we need the position of the moon and the sun at these times.

In [ ]:
from astropy.coordinates import get_moon, get_sun
from astropy.time import Time
times = Time(data['jd'], format='jd')   # makes an array of astropy.Time objects
moon = get_moon(times)                  # makes an array of moon positions
sun = get_sun(times)                    # makes an array of sun positions

Currently, `astropy.coordinates` does not have a lunar phase function, so we'll just use the angular separation between the sun and moon as a proxy. If the angular separation is 0 degrees, that's new moon, whereas an angular separation of 180 degrees is full moon. Other phases lie in between. `moon` and `sun` are arrays of `SkyCoord` objects that have many useful tools for computing sky posisitions. Here we'll use the `separation()` function, which computes the angular separation on the sky between two objects:

In [ ]:
seps = moon.separation(sun)                         # angular separation from moon to sun
data['phase'] = pd.Series(seps, index=data.index)   # Add this new parameter to the data frame

Now that we have the phase information, let's see if our earlier hypothesis about the moon being a source of background light is valid. We'll plot one versus the other, again using the `pandas` built-in plotting functionality.

In [ ]:
data.plot.scatter('phase','magsky')

Great! There's a definite trend there, but also some interesting patterns. Remember these are magnitudes per square arc-second, so brighter sky is down, not up. We can also split up the data based on the phase and plot the resulting histograms together. You can run this next snippet of code with different `phasecut` values to see how they separate out. We use `matplotlib`'s `gca` function to "get the current axis", allowing us to over-plot two histograms.

In [ ]:
import matplotlib.pyplot as plt
phasecut = 90.
res = data[data.phase>phasecut].hist('magsky', bins=50, label='> {:.2f} degrees'.format(phasecut), alpha=0.7)
ax = plt.gca()
res = data[data.phase<phasecut].hist('magsky', ax=ax, bins=50, label='< {:.2f} degrees'.format(phasecut), alpha=0.7)
plt.legend(loc='upper left')

Success! It definitely looksl like scattered moonlight is responsible for the bulk of the added sky brightness. But there's also a portion of data where the moon was bright but the sky was still dark. There's more to it than just phase. Now we turn to the task of fitting a model to this.

### 2)  The Model

Turns out that the definitive reference for this was authored by a colleague of mine:  Kevin Krisciunas at Texas A&M. His paper can be found at the ADS abstract service:  http://adsabs.harvard.edu/abs/1991PASP..103.1033K

You can read the details (lots of empirical formulas, light-scattering theory, and unit conversions), but the short of it is that we get a predictive model of the sky-brightness at the position of an astronomical object as a function of the following variables:

1. The lunar phase angle: $\alpha$
2. The angular separation between the object and the moon: $\rho$
3. The Zenith angle of the object:  $Z$
4. The Zenith angle of the moon:  $Z_m$
5. The extinction coefficient:  $k_X$ (a measure of how much the atmosphere absorbs light)
6. The dark-sky (no moon) sky background at zenith (in mag/square-arc-sec):  $m_{dark}$

The following diagram shows some of these variables:  ![diagram showing variables](media/Embed.jpeg)

Actually, $\alpha$, $\rho$, $Z$, and $Z_m$ are all functions of the date of observations and sky coordinates of the object, which we have already. That leaves $k_x$ and $m_{dark}$ as the only unknowns to be determined. Given these variables, the flux from the moon is given by an empirically-determined function that takes into account the fact that the moon is not a perfect sphere:

$$I^* = 10^{-0.4(3.84 + 0.026|\alpha | + 4\times 10^{-9}\alpha^4)}$$

This flux is then scattered by angle $\rho$ into our line of sight, contributing to the sky background. The fraction of light scattered into angle $\rho$ is given empirically by:

$$f(\rho) = 10^{5.36}\left[1.06 + \cos^2\rho\right] + 10^{6.15 - \rho/40} $$

This just tells us how quickly the sky brightness falls off as we look further away from the moon. We can visualize this by making a 2D array of angles from the center of an image ($\rho$) and comptuing $f(\rho)$. The first part of the next cell uses numpy array functions to create a 2D "image" with the moon at center and each pixel representing a value of $\rho$ degrees from the center.

In [ ]:
import numpy as np
jj,ii = np.indices((1024,1024))/1024             # 2D index arrays scaled 0->1 
rho = np.sqrt((ii-0.5)**2 + (jj-0.5)**2)*45.0    # 2D array of angles from center in degrees

f = 10**5.36*(1.06 + (np.cos(rho*np.pi/180)**2)) + np.power(10, 6.15-rho/40) 
plt.imshow(f, origin='lower', extent=(-22.5,22.5,-22.5,22.5))
plt.contour(f, origin='lower', extent=(-22.5,22.5,-22.5,22.5), colors='white', alpha=0.1)
plt.xlabel('X angular distance')
plt.ylabel('Y angular distance')

So there's less and less scattered light farther from the moon (at the center). But this scattered light is also attenuated (absorbed) by the atmosphere. This attenuation is parametrized by the *airmass* $X$, the relative amount of atmosphere the light has to penetrate (with $X=1$ for the zenith). Krisciunas & Schaefer (1991) present this formula for the airmass:  $X(Z) = \left(1 - 0.96 \sin^2 Z\right)^{-1/2}$. We'll come back to this later. Suffice it to say for the moment that this is an approximation very close to the "infinite slab" model of the atmosphere. Putting it all together, the surface brigthness (in the interesting units of [nanoLamberts](https://en.wikipedia.org/wiki/Lambert_(unit))) from the moon will be:

$$ B_{moon} = f(\rho)I^*10^{-0.4 k_X X(Z_m)}\left[1 - 10^{-0.4k_X X(Z)}\right] $$

Let's visualize that first factor, which attenuates the light from the moon. I'll just set $I^*=1$ and $k_X=5$ to make the effect obvious. We'll define the airmass function for later use as well. Let's assume the moon is at a zenith angle of 22.5$^\circ$ so the bottom of the graph corresponds to $Z=45^\circ$ and the top is the zenith $Z=0^\circ$. <a id="airmass"></a>

In [ ]:
def X(Z):
    '''Airmass as afunction zenith angle Z in radians'''
    return 1./np.sqrt(1 - 0.96*np.power(np.sin(Z),2))

Z = (45 - jj*45)*np.pi/180.    # rescale jj (0->1) to Z (45->0) and convert to radians
plt.imshow(f*np.power(10, -0.4*5*X(Z)), origin='lower', extent=(-22.5,22.5,45,0))
plt.contour(f*np.power(10, -0.4*5*X(Z)), origin='lower', extent=(-22.5,22.5,45,0), colors='white', alpha=0.1)
plt.xlabel('X angular distance')
plt.ylabel('Zenith angle Z')

So as we get closer to the horizon, there's less moonlight, as it's been attenuated by the larger amount of atmosphere. Lastly, to convert these nanoLamberts into magnitudes per square arc-second, we need the dark (no moon) sky brightness at the zenith, $m_{dark}$, and convert that to nanoLamberts using this formula:

$$ B_{dark} = 34.08\exp (20.7233 - 0.92104 m_{dark})10^{-0.4 k_X (X(Z)-1)}X(Z) $$

where we have also corrected for attenuation by the atmosphere and air-glow (which increases with airmass). The final model for observed sky brightness $m_{sky}$ is:

$$ m_{sky} = m_{dark} - 2.5 \log_{10}\left(\frac{B_{moon} + B_{dark}}{B_{dark}}\right) $$

Whew! That's a lot of math. But that's all it is, and we can make a python function that will do it all for us.

In [ ]:
def modelsky(alpha, rho, kx, Z, Zm, mdark):
    Istar = np.power(10, -0.4*(3.84+0.026*np.absolute(alpha)+4e-9*np.power(alpha,4)))
    frho = np.power(10, 5.36)*(1.06 + np.power(np.cos(rho),2))+np.power(10, 6.15-rho*180./np.pi/40)
    Bmoon = frho*Istar*np.power(10,-0.4*kx*X(Zm))*(1-np.power(10,-0.4*kx*X(Z)))
    Bdark = 34.08*np.exp(20.723 - 0.92104*mdark)*np.power(10,-0.4*kx*(X(Z)-1))*X(Z)
    return mdark - 2.5*np.log10((Bmoon+Bdark)/Bdark)

Note that all angles should be entered in radians to work with `numpy` trig functions. 

### 3) Data Munging

Now, we just need the final ingredients:  $\alpha$, $\rho$, $Z$, and $Z_m$, all of which are computed using `astropy.coordinates`. The lunar phase angle $\alpha$ is defined as the angular separation between the Earth and Sun as observed *on the moon*. Alas, `astropy` can't compute this directly (guess they never thought lunar astronauts would use the software). But since the Earth-moon distance is much less than the Earth-sun distance (i.e., $\gamma \sim 0$), this is close enough to 180 degrees minus the angular separation between the moon and sun as observed on Earth (call it $\beta$, which we already computed). See diaram below. ![Diagram showing Earth, moon, and sun](media/EarthMoonSun.jpg)

In [ ]:
alpha = (180. - data['phase'])      # Note: these need to be in degrees
data['alpha'] = pd.Series(alpha, index=data.index)

Next, in order to compute zenith angles and azimuths, we need to tell the `astropy` functions where on Earth we are located, since these quantities depend on our local horizon. Luckily, Las Campanas Observatory (LCO) is in `astropy`'s database of locations. We'll also need to create locations on the sky for all our background observations.

In [ ]:
from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from astropy import units as u
lco = EarthLocation.of_site('lco')
fields = SkyCoord(data['RA']*u.degree, data['Decl']*u.degree)         # astropy often requires units
f_altaz = fields.transform_to(AltAz(obstime=times, location=lco))     # Transform from RA/DEc to Alt/Az
m_altaz = moon.transform_to(AltAz(obstime=times, location=lco))

rho = moon.separation(fields)*np.pi/180.0   # angular distance between moon and all fields
Z = (90. - f_altaz.alt.value)*np.pi/180.0   # remember:  we need things in radians
Zm = (90. - m_altaz.alt.value)*np.pi/180.0 
skyaz = f_altaz.az.value
data['rho'] = pd.Series(rho, index=data.index)
data['Z'] = pd.Series(Z, index=data.index)     # radians
data['Zm'] = pd.Series(Zm, index=data.index)
data['skyaz'] = pd.Series(skyaz, index=data.index)

I've added the variables to the Pandas `dataFrame` as it will help with plotting later. We can try plotting some of these variables against others to see how things look.  Let's try a scatter plot of moon/sky separation vs. sky brightness and color the points according to lunar phase. I tried this with the Pandas `scatter()` and it didn't look that great, so we'll do it with the matplotlib functions directly. Also with `matplotlib` we can invert the y axis so that brighter is 'up'.

In [ ]:
fig,axes = plt.subplots(1,2, figsize=(15,6))
sc = axes[0].scatter(data['rho'], data['magsky'], marker='.', c=data['alpha'], cmap='viridis_r')
axes[0].set_xlabel(r'$\rho$', fontsize=16)
axes[0].set_ylabel('Sky brightness (mag/sq-arc-sec)', fontsize=12)
axes[0].text(1.25, 0.5, "lunar phase", va='center', ha='right', rotation=90,
            transform=axes[0].transAxes, fontsize=12)
axes[0].invert_yaxis()
fig.colorbar(sc, ax=axes[0])
sc = axes[1].scatter(data['alpha'], data['magsky'], marker='.', c=data['rho'], cmap='viridis_r')
axes[1].set_xlabel('Lunar phase', fontsize=12)
axes[1].set_ylabel('Sky brightness (mag/sq-arc-sec)', fontsize=12)
axes[1].text(1.25, 0.5, r"$\rho$", va='center', ha='right', rotation=90,
            transform=axes[1].transAxes, fontsize=12)
axes[1].invert_yaxis()
ymin,ymax = axes[0].get_ylim()
fig.colorbar(sc, ax=axes[1])


There certainly seems to be a trend that the closer to full ($\alpha = 0$, yellow), the brighter the background and the closer the moon is to the field (lower $\rho$), the higher the background. Looks good. 

### 4) Fitting (Training) the Model

Let's try and fit this data with our model and solve for $m_{dark}$, and $k_x$, the only unknowns in the problem. For this we need to create a dummy function that we can use with `scipy`'s `leastsq` function. It needs to take a list of parameters (`p`) as its first argument, followed by any other arguments and return the weighted difference between the model and data. We don't have any weights (uncertainties), so it will just return the differences.

In [ ]:
from scipy.optimize import leastsq
def func(p, alpha, rho, Z, Zm, magsky):
    mdark,kx = p
    return magsky - modelsky(alpha, rho, kx, Z, Zm, mdark)

We now run the least-squares function, which will find the parameters `p` which minimize the squared sum of the residuals (i.e. $\chi^2$). `leastsq` takes as arguments the function we wrote above, `func`, an initial guess of the parameters, and a tuple of extra arguments needed by our function. It returns the best-fit parameters and a status code. We can print these out, but also use them in our `modelsky` function to get the prediction that we can compare to the observed data.

In [ ]:
pars,stat = leastsq(func, [22, 0.2], args=(data['alpha'],data['rho'],data['Z'],data['Zm'],data['magsky']))
print(pars)
# save the best-fit model and residuals
data['modelsky']=pd.Series(modelsky(data['alpha'],data['rho'],pars[1],data['Z'],data['Zm'],pars[0]), index=data.index)
data['residuals']=pd.Series(data['magsky']-data['modelsky'], index=data.index)

Now that we have a model, we have a way to *predict* the sky brightness. So let's make the same two plots as we did above, but this time plotting the *model* brigthnesses rather than the observed brightnesses. Just to see if we get the same kinds of patterns/behaviours. This next cell is a copy of the earlier one, just changing `magsky` into `modelsky`. 

In [ ]:
fig,axes = plt.subplots(1,2, figsize=(15,6))
sc = axes[0].scatter(data['rho'], data['modelsky'], marker='.', c=data['alpha'], cmap='viridis_r')
axes[0].set_xlabel(r'$\rho$', fontsize=16)
axes[0].set_ylabel('Sky brightness (mag/sq-arc-sec)', fontsize=12)
axes[0].text(1.25, 0.5, "lunar phase", va='center', ha='right', rotation=90,
            transform=axes[0].transAxes, fontsize=12)
axes[0].invert_yaxis()
fig.colorbar(sc, ax=axes[0])
sc = axes[1].scatter(data['alpha'], data['modelsky'], marker='.', c=data['rho'], cmap='viridis_r')
axes[1].set_xlabel('Lunar phase', fontsize=12)
axes[1].set_ylabel('Sky brightness (mag/sq-arc-sec)', fontsize=12)
axes[1].text(1.25, 0.5, r"$\rho$", va='center', ha='right', rotation=90,
            transform=axes[1].transAxes, fontsize=12)
axes[1].invert_yaxis()
axes[0].set_ylim(ymin,ymax)
axes[1].set_ylim(ymin,ymax)
fig.colorbar(sc, ax=axes[1])

You will see that there are some patterns that are correctly predicted, but others that are not. In particular, there's a whole cloud of points with $\alpha < 0.8$ and sky brightness > 22 that are observed but *not* predicted. In other words, we observed some objects where the moon was relatively bright, yet the sky was relatively dark.

This is where I hit a bit of a wall in my investigation. It was not at all obvious where these points were coming from because the data set was so large and we have so many variables at work. However, by luck this ended up being around the time that Shannon was playing around with [Bokeh](https://docs.bokeh.org/en/latest/index.html) and it turned out to be exactly what I needed to explore where things were not working correctly. Let's do that now.

### 5) Plotting Residuals
A good way to see where a model is failing is to plot the residuals (observed - model). Where the residuals are close to zero, the model is doing a good job, but where the residuals are large (positive or nagative), the model is failing to capture something. A good diagnostic is to plot these residuals versus each of your variables and see where things go wrong. The great thing about Bokeh is it gives a very powerful way to do this:  linking graphs so that selecting points in one graph will select the corresponding points in all other graphs that share the same dataset. This is why we've been adding our variables to the pandas `dataFrame`, `data`:  that's what Bokeh uses for plotting. In this code block we setup a Bokeh graph and plot 6 different "slices" through our multi-dimenisonal data. In the resulting plots, try selecting different regions of the upper-left panel (the residuals) to see if they correspond to interesting sets of parameters in the other panels.

In [ ]:
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.io import show,output_notebook
from bokeh.models import ColumnDataSource

output_notebook()
source = ColumnDataSource(data)
TOOLS = ['box_select','lasso_select','reset','box_zoom','help']
vars = [('alpha','residuals'),('alpha','rho'),('alpha','Zm'),
        ('jd','alpha'),('Z','Zm'),('RA','Decl')]
plots = []
for var in vars:
   s = figure(tools=TOOLS, plot_width=300, plot_height=300)
   s.circle(*var, source=source, selection_color='red')
   s.xaxis.axis_label = var[0]
   s.yaxis.axis_label = var[1]
   plots.append(s)
#plots[0].line([17.8,22.3],[17.8,22.3], line_color='orangered')

p = gridplot([plots[0:3],plots[3:]])
show(p)

With a little data exploring, it's pretty obvious that the majority of the outlying points comes from observations when the moon is relatively full but very low (or even below) the horizon. The reason is that the airmass formula that we implemented above has a problem with $Zm > \pi/2$. To see this, we can simply plot `X(Z)` as a function of 'Z':

In [ ]:
from matplotlib.pyplot import plot, xlabel, ylabel,ylim
Z = np.linspace(0, 3*np.pi/4, 100)    # make a range of Zenith angles
plot(Z*180/np.pi, X(Z), '-')
xlabel('Zenith angle (degrees)')
ylabel('Airmass')

So the airmass (amount of air the light travels through) increases as you get to the horizon ($Z=90^\circ$), but then decreases. That's not right! This is the reason the model is failing for some points. Can you think of a way to easily fix this problem? Try it out. Just [go back](#airmass) to the cell above where `X(Z)` is defined and change it. Then select `Cell -> Run All Below` from the menu so see how the results change. There's also an entire [Wikipedia page](https://en.wikipedia.org/wiki/Air_mass_(astronomy)) with many airmass approximations and formulae, which you could try coding and seeing if they work better.

### 6) Final Remarks

At this point you might be feeling overwhelmed. How did I know which modules to use? How did I know how to use them? The answer:  Google, ADS, and 20+ years (eek!) of experience coding in Python. I also neglected to show all the dead-ends and mistakes I made on the way to getting the final solution, all the emails I sent to Kevin asking about the details of his paper, and trips to Shannon's office to get help with using Bokeh.

Before you start tackling a particular problem it's well worth your time to research whether there is already a solution "out there" that you can use or modify for your use. It has never been so easy to do this, thanks to search engines ([Google](https://www.google.com), et al.), data/software catalogs ([PyPI](https://pypi.org), et al.), discussion groups ([Stackoverflow](https://stackoverflow.com/), et al.) and even social media ([python users in astronomy facebook group](https://www.facebook.com/groups/astropython/), etc). And your friendly neighborhood python experts are there to make helpful suggestions.

Don't re-invent the wheel, but improve it by all means.